In [2]:
#Exploratory data analysis notebook

import os
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from icecream import ic
import sys
sys.path.append(parent_directory)
from libs.preprocessor import Preprocessor
from libs.utils import *
from libs.data_prep import *

Current Directory: /Users/cisilkaraguzel/Documents/GitHub/AD_detection_DL_approach/notebooks
Parent Directory: /Users/cisilkaraguzel/Documents/GitHub/AD_detection_DL_approach


# Dataset Information

Our dataset is from ADNI, with following properties: 

- 3 plane localizer

- T1

- Axial

In [3]:
df_AD = pd.read_csv("../data-ADNI/interim/All_AD_3loc_T1_axial_3_26_2024.csv")
new_df_AD= remove_duplicates(df_AD)

data_folder = "../data-ADNI/raw/All_AD_3loc_T1_axial/ADNI"  
data_paths = get_filtrated_data_paths(data_folder, new_df_AD["Image Data ID"].tolist())

In [5]:
new_df_AD

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
59,I116116,002_S_0619,AD,M,80,m24,MRI,3-plane localizer,Original,2008-08-13,DCM,3/26/2024
193,I88800,002_S_0816,AD,M,72,m12,MRI,3-plane localizer,Original,2008-01-28,DCM,3/26/2024
101,I124839,002_S_0938,AD,F,84,m24,MRI,3-plane localizer,Original,2008-10-31,DCM,3/26/2024
149,I54256,002_S_0955,AD,F,79,m06,MRI,3-plane localizer,Original,2007-05-10,DCM,3/26/2024
61,I130785,002_S_1018,AD,F,73,m24,MRI,3-plane localizer,Original,2008-12-12,DCM,3/26/2024
...,...,...,...,...,...,...,...,...,...,...,...,...
72,I109464,137_S_0366,AD,F,59,m24,MRI,3-plane localizer,Original,2008-06-12,DCM,3/26/2024
46,I14220,137_S_0438,AD,M,82,sc,MRI,3-plane localizer,Original,2006-04-27,DCM,3/26/2024
171,I136025,137_S_0796,AD,M,83,m24,MRI,3-plane localizer,Original,2009-02-12,DCM,3/26/2024
189,I51632,137_S_0841,AD,F,78,m06,MRI,3-plane localizer,Original,2007-04-26,DCM,3/26/2024


In [14]:
new_df_AD["Subject"].unique

<bound method Series.unique of 59     002_S_0619
193    002_S_0816
101    002_S_0938
149    002_S_0955
61     002_S_1018
          ...    
72     137_S_0366
46     137_S_0438
171    137_S_0796
189    137_S_0841
89     137_S_1041
Name: Subject, Length: 73, dtype: object>

In [9]:
print("There are "+str(len(new_df_AD))+ " subjects with AD in our dataset.")

There are 73 subjects with AD in our dataset.


In [15]:
df_CN = pd.read_csv("../data-ADNI/interim/All_CN_3loc_T1_axial_3_26_2024.csv")
new_df_CN= remove_duplicates(df_CN)

data_folder_CN = "../data-ADNI/raw/All_CN_3loc_T1_axial/ADNI"
data_paths_CN = get_filtrated_data_paths(data_folder_CN, new_df_CN["Image Data ID"].tolist())

new_df_CN

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
8,I173728,002_S_0295,CN,M,89,m48,MRI,3-plane localizer,Original,2010-05-13,DCM,3/22/2024
5,I173458,002_S_0413,CN,F,80,m48,MRI,3-plane localizer,Original,2010-05-06,DCM,3/22/2024
78,I147109,002_S_0559,CN,M,82,m36,MRI,3-plane localizer,Original,2009-06-30,DCM,3/22/2024
409,I185098,002_S_0685,CN,F,94,m48,MRI,3-plane localizer,Original,2010-07-15,DCM,NaN
34,I166840,002_S_1261,CN,F,74,m36,MRI,3-plane localizer,Original,2010-02-25,DCM,3/22/2024
...,...,...,...,...,...,...,...,...,...,...,...,...
102,I142412,137_S_0283,CN,M,82,m36,MRI,3-plane localizer,Original,2009-04-23,DCM,3/22/2024
440,I229300,137_S_0301,CN,F,79,v06,MRI,3-plane localizer,Original,2011-04-27,DCM,NaN
90,I143199,137_S_0459,CN,M,76,m36,MRI,3-plane localizer,Original,2009-05-05,DCM,3/22/2024
62,I150899,137_S_0686,CN,M,75,m36,MRI,3-plane localizer,Original,2009-07-29,DCM,3/22/2024


In [16]:
new_df_CN["Subject"].unique

<bound method Series.unique of 8      002_S_0295
5      002_S_0413
78     002_S_0559
409    002_S_0685
34     002_S_1261
          ...    
102    137_S_0283
440    137_S_0301
90     137_S_0459
62     137_S_0686
468    137_S_0972
Name: Subject, Length: 95, dtype: object>

In [17]:
print("There are "+str(len(new_df_CN))+ " subjects with CN in our dataset.")

There are 95 subjects with CN in our dataset.


In [19]:
df_MCI = pd.read_csv("../data-ADNI/interim/All_MCI_3loc_T1_axial_3_26_2024.csv")
new_df_MCI= remove_duplicates(df_MCI)

data_folder_MCI = "../data-ADNI/raw/All_MCI_3loc_T1_axial/ADNI"
data_paths_MCI = get_filtrated_data_paths(data_folder_MCI, new_df_MCI["Image Data ID"].tolist())

new_df_MCI

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
682,I186075,002_S_0729,MCI,F,69,m48,MRI,3-plane localizer,Original,2010-07-22,DCM,NaN
625,I154515,002_S_0782,MCI,M,85,m36,MRI,3-plane localizer,Original,2009-09-04,DCM,NaN
456,I108595,002_S_0954,MCI,F,71,m18,MRI,3-plane localizer,Original,2008-06-05,DCM,NaN
641,I160749,002_S_1070,MCI,M,77,m36,MRI,3-plane localizer,Original,2009-12-10,DCM,NaN
706,I209893,002_S_1155,MCI,M,62,m48,MRI,3-plane localizer,Original,2011-01-06,DCM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
343,I79417,137_S_0825,MCI,M,88,m12,MRI,3-plane localizer,Original,2007-10-29,DCM,NaN
700,I207428,137_S_0973,MCI,M,81,m48,MRI,3-plane localizer,Original,2010-12-03,DCM,NaN
750,I270361,137_S_0994,MCI,F,60,v06,MRI,3-plane localizer,Original,2011-12-06,DCM,NaN
738,I254366,137_S_1414,MCI,M,79,v06,MRI,3-plane localizer,Original,2011-09-02,DCM,NaN


In [20]:
new_df_MCI["Subject"].unique

<bound method Series.unique of 682    002_S_0729
625    002_S_0782
456    002_S_0954
641    002_S_1070
706    002_S_1155
          ...    
343    137_S_0825
700    137_S_0973
750    137_S_0994
738    137_S_1414
524    137_S_1426
Name: Subject, Length: 153, dtype: object>

In [21]:
print("There are "+str(len(new_df_MCI))+ " subjects with MCI in our dataset.")

There are 153 subjects with MCI in our dataset.
